# Labelling with OpenAI GPT 5 Nano

## Install packages

In [24]:
%pip install openai python-dotenv pandas

Note: you may need to restart the kernel to use updated packages.


## Import packages

In [221]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import pandas as pd
import json
import time
import math
import gzip

## Login to OpenAI

In [111]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

## Load data

### kaggle

In [232]:
df = pd.read_csv("./data/reviews.csv")
df = df.dropna(subset=["text"]).reset_index(drop=True)
print(len(df))
print(df.head())

1100
                     business_name    author_name  \
0  Haci'nin Yeri - Yigit Lokantasi    Gulsum Akar   
1  Haci'nin Yeri - Yigit Lokantasi  Oguzhan Cetin   
2  Haci'nin Yeri - Yigit Lokantasi     Yasin Kuyu   
3  Haci'nin Yeri - Yigit Lokantasi     Orhan Kapu   
4  Haci'nin Yeri - Yigit Lokantasi     Ozgur Sati   

                                                text  \
0  We went to Marmaris with my wife for a holiday...   
1  During my holiday in Marmaris we ate here to f...   
2  Prices are very affordable. The menu in the ph...   
3  Turkey's cheapest artisan restaurant and its f...   
4  I don't know what you will look for in terms o...   

                                               photo  rating  \
0         dataset/taste/hacinin_yeri_gulsum_akar.png       5   
1        dataset/menu/hacinin_yeri_oguzhan_cetin.png       4   
2  dataset/outdoor_atmosphere/hacinin_yeri_yasin_...       3   
3  dataset/indoor_atmosphere/hacinin_yeri_orhan_k...       5   
4           dataset

### UCSD

In [239]:
def parse(path):
    with gzip.open(path, 'rt', encoding='utf-8') as g:
        for line in g:
            yield json.loads(line)
data = list(parse("./data/review-Vermont_10.json.gz"))
df = pd.DataFrame(data)
df = df.dropna(subset=["text"]).reset_index(drop=True)
print(len(df))
print(df.head())

178168
                 user_id             name           time  rating  \
0  118026874392842649478    rebecca kerns  1620085852324       5   
1  101532740754036204131    Peter DeForge  1580309946474       5   
2  115404122636203550540    Chad Goulette  1605195974445       5   
3  104789336434407408181  Mark LaFountain  1593005848256       5   
4  108980665975608069965           Jeff R  1582059996120       5   

                                                text  pics  \
0      Always done right from wood stove to screens!  None   
1  A great company to work with.  Their sales and...  None   
2  Great place to do business with staff was grea...  None   
3  Awesome Customer service, quick response, and ...  None   
4  If you need a top quality job, by a group of p...  None   

                                                resp  \
0  {'time': 1620087641504, 'text': 'Good Evening,...   
1  {'time': 1580320228721, 'text': 'Good Afternoo...   
2  {'time': 1605195166792, 'text': 'Hi Chad

## Creating prompt

In [236]:
SYSTEM_PROMPT_STRING = """You are a strict moderation judge for location reviews. Classify each review into exactly ONE of:
advertisement (self-promo, discount codes, contact links),
irrelevant (off-topic, questions/chat unrelated to a real visit),
rant_without_visit (angry/defamatory without evidence of an actual visit),
clean (a normal on-topic review—positive or negative—from a real/likely visit).
If multiple seem plausible, choose the most severe (advertisement > irrelevant > rant_without_visit > clean).
Ignore emojis, casing, punctuation spam, and translation artifacts.
Do not guess facts."""

schema = {
    "name": "google_reviews_label",
    "schema": {
        "type": "object",
        "properties": {
            "label": {"type":"string","enum":["advertisement","irrelevant","rant_without_visit","clean"]}
        },
        "required": ["label"],
        "additionalProperties": False
    }
}

df["prompt"] = df["text"].apply(lambda review: f"Classify this review:\n<review>{review}</review>")
print(df["prompt"].head())

0    Classify this review:\n<review>Always done rig...
1    Classify this review:\n<review>A great company...
2    Classify this review:\n<review>Great place to ...
3    Classify this review:\n<review>Awesome Custome...
4    Classify this review:\n<review>If you need a t...
Name: prompt, dtype: object


## Labelling

In [238]:
jsonl_file = "./data/requests.jsonl"
responses_file = "./data/responses.jsonl"
if os.path.exists(jsonl_file):
    os.remove(jsonl_file)
if os.path.exists(responses_file):
    os.remove(responses_file)
for i in range(math.ceil(len(df)/750)):
    with open(jsonl_file, "w", encoding="utf-8") as f:
        for j, prompt in enumerate(df["prompt"][i * 750:(i + 1) * 750], start=1):
            request_obj = {
                "custom_id": f"{i*750 + j}",
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                "model": "gpt-5-nano",
                "messages": [
                        {"role":"system","content": SYSTEM_PROMPT_STRING},
                        {"role":"user","content": prompt}
                    ]
                }
            }
            f.write(json.dumps(request_obj) + "\n")
    batch_input_file = client.files.create(
        file=open(jsonl_file, "rb"),
        purpose="batch"
    )
    batch_input_file_id = batch_input_file.id
    batch = client.batches.create(
        input_file_id=batch_input_file_id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={
            "description": "nightly eval job"
        }
    )
    batch_id = batch.id
    batch = client.batches.retrieve(batch_id)
    while (batch.status != "completed"):
        time.sleep(5)
        batch = client.batches.retrieve(batch_id)
    file_response = client.files.content(batch.output_file_id)
    with open(responses_file, "a", encoding="utf-8") as f:
        f.write(file_response.text)

## Save to csv file

In [252]:
df_responses = pd.read_json(responses_file, lines=True)
df["label"] = df_responses["response"].apply(lambda x: x['body']['choices'][0]['message']['content'])
df.to_csv("./data/review-Vermont_10-labeled.csv", index=False)
print(df.head())

                 user_id             name           time  rating  \
0  118026874392842649478    rebecca kerns  1620085852324       5   
1  101532740754036204131    Peter DeForge  1580309946474       5   
2  115404122636203550540    Chad Goulette  1605195974445       5   
3  104789336434407408181  Mark LaFountain  1593005848256       5   
4  108980665975608069965           Jeff R  1582059996120       5   

                                                text  pics  \
0      Always done right from wood stove to screens!  None   
1  A great company to work with.  Their sales and...  None   
2  Great place to do business with staff was grea...  None   
3  Awesome Customer service, quick response, and ...  None   
4  If you need a top quality job, by a group of p...  None   

                                                resp  \
0  {'time': 1620087641504, 'text': 'Good Evening,...   
1  {'time': 1580320228721, 'text': 'Good Afternoo...   
2  {'time': 1605195166792, 'text': 'Hi Chad!

Tha.

## Change label to 0 and 1

In [255]:
df = pd.read_csv("./data/review-Vermont_10-labeled.csv")
df['label'] = df['label'].apply(lambda x: 0 if str(x).lower() == "clean" else 1)
df = df[['text', 'label']]
df.to_csv("./data/review-Vermont_10-binary.csv", index=False)
print(df.head())

C:\Users\Moey Sean Jean\AppData\Local\Temp\ipykernel_9484\1943313882.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data/review-Vermont_10-labeled.csv")


                                                text  label
0      Always done right from wood stove to screens!      0
1  A great company to work with.  Their sales and...      0
2  Great place to do business with staff was grea...      0
3  Awesome Customer service, quick response, and ...      0
4  If you need a top quality job, by a group of p...      0
